In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/My Drive/ModelSharing'

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

The function series_to_supervised takes several arguments: data (the input dataset), n_in (the number of lag observations as input), n_out (the number of future observations to predict), and dropnan (a flag indicating whether to drop rows with missing values or not).

- It determines the number of variables in the dataset (n_vars) based on whether the data is a list or a DataFrame.

- It creates a DataFrame (df) from the input dataset.

- Two empty lists, cols and names, are initialized. These lists will store the columns and column names for the transformed dataset.

- The next block of code handles the input sequence, which consists of lag observations. It iterates over the range of n_in to 0 (exclusive) in reverse order. For each iteration, it appends a shifted version of the DataFrame by i time steps to the cols list. It also adds the corresponding column names to the names list.

The following block handles the forecast sequence, which represents future observations to predict. It iterates over the range of 0 to n_out. For each iteration, it appends a shifted version of the DataFrame in the negative direction (to predict future values). If it's the first iteration (i.e., i == 0), it adds column names indicating the current time step. Otherwise, it adds column names indicating future time steps.

- The cols list is concatenated along the columns axis (axis=1) to create a new DataFrame called agg.

- The column names of agg are assigned from the names list.

- If the dropnan flag is set to True, the function drops rows with missing values from agg.

- Finally, the transformed dataset (agg) is converted to a DataFrame of type 'float32' and returned.

In summary, this function takes a dataset and transforms it into a supervised learning format by creating lag observations as input and future observations as output. It can be useful for tasks such as time series forecasting, where we want to predict future values based on past observations.

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=False):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
  # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
  # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return pd.DataFrame(agg.astype('float32'))

The function train_valid_test_split takes four arguments: data (the input dataset), hours_of_history (the number of hours of historical data to include as input), hours_to_predict (the number of hours to predict in the future), and parameters_included (a list of parameters/features to include in the dataset).

- The first line of code selects the first 52,608 rows of the dataset as data_train_valid, representing the first 6 years of data for training and validation.

- The second line of code selects the remaining rows of the dataset as data_test, representing the last 1 year of data for testing.

- The next two lines of code remove any rows with missing values from data_train_valid and data_test by calling the dropna method.

- The train_test_split function is used to split data_train_valid into two sets: data_valid and data_train. The test_size parameter is set to 0.4, indicating that 40% of the data will be used for validation, while the remaining 60% will be used for training. The shuffle parameter is set to False, indicating that the data will not be randomly shuffled before splitting.

Finally, the function returns the values of data_train, data_valid, and data_test as NumPy arrays.

In [5]:
def train_valid_test_split(data, hours_of_history, hours_to_predict, parameters_included):
    data_train_valid = data.iloc[:52608,:] # the first 6 years for training/validation
    data_test = data.iloc[52608:,:] # the last 1 years for test evaluation
    data_train_valid.dropna(inplace=True)
    data_test.dropna(inplace=True)

    data_valid, data_train = train_test_split(data_train_valid, test_size=0.4, shuffle= False) # the last 60% data in the first 6 years used for training and the first 40% used for validation.

    return data_train.values, data_valid.values, data_test.values

This code defines a function called prepare_data that prepares the data for a specific station by performing several preprocessing steps. Let's go through the code:

1. The function prepare_data takes four arguments: station_id (the ID of the station), hours_of_history (the number of hours of historical data to include as input), hours_to_predict (the number of hours to predict in the future), and parameters_included (a list of parameters/features to include in the dataset).

2. The next line reads the data from a CSV file specific to the station using the pd.read_csv function. It removes the first column (assuming it contains index values) using .iloc[:,1:].

3. The data is then scaled using min-max scaling (MinMaxScaler) with the fit method applied to the training and validation data (data.iloc[:52608,:]). This ensures that the scaling is performed without including the test dataset.

4. The maximum and minimum values of the third column (assuming it represents discharge) are calculated to later use for reference (q_max and q_min).

5. The series_to_supervised function is called to convert the scaled data into a supervised learning format, considering the specified hours_of_history and hours_to_predict.

6. The train_valid_test_split function is called to split the data sequence into training, validation, and test sets based on the provided parameters.

7. The training data is then split into separate arrays for different input features: train_x_rainfall, train_x_discharge, and train_x_et. The train_discharge array is reshaped to include the required hours of history and prediction.

8. Similarly, the validation and test data are split into separate arrays for input features and output labels.

9. The function returns a list of arrays containing the training inputs (train_x_et, train_x_discharge, train_x_rainfall), the training output labels (train_y), the validation inputs and labels, the test inputs and labels, and the maximum and minimum discharge values (q_max and q_min).

In summary, this function reads and scales the data, converts it into a supervised learning format, and splits it into training, validation, and test sets. It also separates the data into different arrays based on the input features and output labels.

In [6]:
def prepare_data(station_id, hours_of_history, hours_to_predict, parameters_included):

    data = pd.read_csv('./data/'+str(station_id)+'_data.csv').iloc[:,1:]

  # simple min-max scaling. Other pretreatments such as normalization also work.
    scaler = MinMaxScaler()
    scaler.fit(data.iloc[:52608,:]) # min-max scaling without the test dataset.
    q_max = np.max(data.iloc[:52608,2]) # manually check the maximum and minimum discharge
    q_min = np.min(data.iloc[:52608,2])
    data_scaled = scaler.transform(data)

  # data split
    data_sequence = series_to_supervised(data_scaled, hours_of_history, hours_to_predict)
    data_train, data_valid, data_test = train_valid_test_split(data_sequence, hours_of_history, hours_to_predict, parameters_included)

  # Split data into 2 parts for encoder (history) and decoder(future).
    train_x_rainfall = data_train[:,0::3].reshape(-1, hours_of_history+hours_to_predict, 1)
    train_discharge = data_train[:,2::3].reshape(-1, hours_of_history+hours_to_predict, 1)
    train_x_discharge = train_discharge[:,:hours_of_history,:]
    train_y = train_discharge[:,hours_of_history:,:]
    train_x_et = data_train[:,3*hours_of_history+1].reshape(-1, 1) # the current hour et

    valid_x_rainfall = data_valid[:,0::3].reshape(-1, hours_of_history+hours_to_predict, 1)
    valid_discharge = data_valid[:,2::3].reshape(-1, hours_of_history+hours_to_predict, 1)
    valid_x_discharge = valid_discharge[:,:hours_of_history,:]
    valid_y = valid_discharge[:,hours_of_history:,:]
    valid_x_et = data_valid[:,3*hours_of_history+1].reshape(-1, 1) # the current hour et

    test_x_rainfall = data_test[:,0::3].reshape(-1, hours_of_history+hours_to_predict, 1)
    test_discharge = data_test[:,2::3].reshape(-1, hours_of_history+hours_to_predict, 1)
    test_x_discharge = test_discharge[:,:hours_of_history,:]
    test_y = test_discharge[:,hours_of_history:,:]
    test_x_et = data_test[:,3*hours_of_history+1].reshape(-1, 1) # the current hour et

    return [train_x_et, train_x_discharge, train_x_rainfall], train_y, [valid_x_et, valid_x_discharge, valid_x_rainfall], valid_y, [test_x_et, test_x_discharge, test_x_rainfall], test_y, q_max, q_min

The code provided defines a custom loss function called nseloss, which stands for Nash-Sutcliffe Efficiency. This loss function measures the normalized squared difference between the true values (y_true) and the predicted values (y_pred). The formula for the nseloss is as follows:

In [7]:
# define custome loss function (you can use the simple 'mse' as well)
def nseloss(y_true, y_pred):
    return K.sum((y_pred-y_true)**2)/K.sum((y_true-K.mean(y_true))**2)

The provided code defines a function called EDLSTM that constructs an Encoder-Decoder LSTM (EDLSTM) model for a time series prediction task. Here's an explanation of the code:

1. The function EDLSTM takes three arguments: hours_of_history (the number of hours of historical data to include as input), hours_to_predict (the number of hours to predict in the future), and parameters_included (a list of parameters/features to include in the model).

2. The code begins by defining the input layers for the three types of data: runoff observation (input_1), rainfall observation and forecast (input_2), and other non-timeseries data (input_phys).

3. The LSTM encoder layers are defined for the runoff observation and rainfall observation/forecast inputs. Both LSTM layers have 256 units and return sequences set to False, indicating that they only output the last hidden state of the LSTM.

4. The concatenate function is used to combine the input_phys, LSTM1, and LSTM2 layers into a single tensor.

5. The RepeatVector layer is used to repeat the combined tensor along the time dimension to match the hours_to_predict.

6. The LSTM decoder layer is defined with 512 units and return sequences set to True.

7. The fully-connected dense layers are defined using a loop. The dim_dense list specifies the dimensions of each dense layer. For each dimension, a TimeDistributed dense layer with a ReLU activation function and a dropout layer with a dropout rate of 0.2 is applied.

8. The final output layer is a TimeDistributed dense layer with a single unit and a ReLU activation function. The output is flattened to a one-dimensional array.

9. The model is created using the Model class, with the defined inputs and main_out as the outputs.

10. Finally, the constructed model is returned.

This code builds an EDLSTM model that takes multiple inputs (runoff observation, rainfall observation/forecast, and other non-timeseries data) and predicts a single output. The model utilizes LSTM layers for encoding and decoding the temporal information and fully-connected dense layers for learning complex relationships in the data.

In [8]:
def EDLSTM(hours_of_history, hours_to_predict, parameters_included):

  # design network
  # input of runoff observation, LSTM encoder
    input_1 = Input(shape=(hours_of_history, 1), name='LSTM1_input') # shape should be 72*1 for runoff observation
    LSTM1 = LSTM(256, return_sequences=False)(input_1)

  # input of rainfall observation and forecast, LSTM encoder
    input_2 = Input(shape=((hours_of_history+hours_to_predict), 1), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for rainfall and additional stations (if there is no upstream station, n=1)
    LSTM2 = LSTM(256, return_sequences=False)(input_2)

  # input of other non-timeseries data, such as daily or monthly data.
    input_phys = Input(shape=(1,), name='phys_input') # one single value of ET. shape = 1.

  # connect all data
    x = concatenate([input_phys, LSTM1, LSTM2]) # Get state vector.
    x = RepeatVector(hours_to_predict)(x) # 24 is the output time dimension

  # LSTM decoder
    x = LSTM(512, return_sequences=True)(x)

  # define fully-connected dense layers
    dim_dense=[512, 256, 256, 128, 64]

  # final fully-connected dense layer for final result
    for dim in dim_dense:
        x = TimeDistributed(Dense(dim, activation='relu'))(x)
        x = TimeDistributed(Dropout(0.2))(x) # Some dropout for dense layers. Some paper mentioned that it is not recommend to have dropout between the RNN/LSTM/GRU layers. Thus, I only apply dropout in the dense layer.
    
    main_out = TimeDistributed(Dense(1, activation='relu'))(x) # here relu provides the final output non-negative, which is corrosponding to my min-max pre-prossing.
    main_out = Flatten()(main_out)

    model = Model(inputs=[input_phys, input_1, input_2], outputs=main_out)
  
    return model

In [9]:
# identify KGE, NSE for evaluation
def nse(y_true, y_pred):
    return 1-np.sum((y_pred-y_true)**2)/np.sum((y_true-np.mean(y_true))**2)
  
def kge(y_true, y_pred):
    kge_r = np.corrcoef(y_true,y_pred)[1][0]
    kge_a = np.std(y_pred)/np.std(y_true)
    kge_b = np.mean(y_pred)/np.mean(y_true)
    return 1-np.sqrt((kge_r-1)**2+(kge_a-1)**2+(kge_b-1)**2)

In [10]:
def main():
  
  # parameters
    station_id = 521
    hours_to_predict = 24
    hours_of_history = 72
    parameters_included = 3

    batch_size = 64
    lr = 0.0001
    epochs = 300
    test_name = './'+str(station_id)+'_model1_'

  # load data
    x_train, y_train, x_valid, y_valid, x_test, y_test, q_max, q_min = prepare_data(station_id, hours_of_history, hours_to_predict, parameters_included)
    model1 = EDLSTM(hours_of_history, hours_to_predict, parameters_included)

  # compile settings
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
    earlystoping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(test_name+'model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    optimizer = RMSprop(lr=lr)
  
    model1.compile(optimizer=optimizer, loss=nseloss) # I used the build-in RMSprop, loss function is 1-NSE. You can use 'mse', 'mae' as well.

    # train model
    history = model1.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(x_valid, y_valid), callbacks=[reduce_lr, earlystoping, checkpoint], verbose=1)

  # save training loss
    loss_train = history.history['loss']
    loss_valid = history.history['val_loss']
    loss_train = pd.DataFrame({'TrainLoss':loss_train})
    loss_valid = pd.DataFrame({'TestLoss':loss_valid})
    LossEpoches = pd.concat([loss_train, loss_valid], axis=1)
    LossEpoches.to_csv(test_name+'loss.csv', index = True)

  # Final Test Review
    model1.load_weights(test_name+'model.h5')

    y_model_scaled = model1.predict(x_test)
    y_model = y_model_scaled*(q_max-q_min)+q_min
    y_test = y_test*(q_max-q_min)+q_min

  # hourly evaluation
    NSEs=[]
    KGEs=[]
    
    for x in range(0, 24):
        y_pred = y_model[:,x]
        y_True = y_test[:,x]
        
    NSEs.append(nse(y_True[:,0],y_pred))
    KGEs.append(kge(y_True[:,0],y_pred))  
    
    NSEs=pd.DataFrame(NSEs)
    NSEs.columns = ['NSE_Test']
    KGEs=pd.DataFrame(KGEs)
    KGEs.columns = ['KGE_Test']
    
    eva = pd.concat([NSEs, KGEs], axis=1)
    eva.to_csv(test_name+'eva.csv', index = True)
 

In [11]:
if __name__ == "__main__":
      main()


ValueError: Length mismatch: Expected axis has 288 elements, new values have 219 elements